In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import json

from agent.mcts import ChessMonteCarloTreeSearch
from agent.predict import MovePredictor
from chess_environment.position_parser import ChessPositionParser

import config

np.random.seed(123)

cfg = config.Config()

Using Theano backend.


In [2]:
# Load ECO file of openings

with open(cfg.resources.eco_file) as f:
    ecos = json.load(f)

ecos


[{'c': 'A00',
  'f': 'rn1qkbnr/ppp2ppp/8/3p4/5p2/6PB/PPPPP2P/RNBQK2R w KQkq',
  'id': 2990,
  'm': 'g1h3 d7d5 g2g3 e7e5 f2f4 c8h3 f1h3 e5f4',
  'n': 'Amar Gambit'},
 {'c': 'A00',
  'f': 'rn1qkbnr/ppp2ppp/8/3p4/8/6PB/PPPPP3/RNBQ1RK1 b kq',
  'id': 46,
  'm': 'g1h3 d7d5 g2g3 e7e5 f2f4 c8h3 f1h3 e5f4 e1g1 f4g3 h2g3',
  'n': 'Amar Opening: Gent Gambit'},
 {'c': 'A00',
  'f': 'rnbqkbnr/ppp2ppp/8/3pp3/5P2/6PN/PPPPP2P/RNBQKB1R b KQkq',
  'id': 47,
  'm': 'g1h3 d7d5 g2g3 e7e5 f2f4',
  'n': 'Amar Opening: Paris Gambit'},
 {'c': 'A00',
  'f': 'r1bqkb1r/ppp2ppp/2np1n2/4p3/2P5/1PN1P3/P2P1PPP/R1BQKBNR w KQkq',
  'id': 2994,
  'm': 'e2e3 e7e5 c2c4 d7d6 b1c3 b8c6 b2b3 g8f6',
  'n': 'Amsterdam Attack'},
 {'c': 'A00',
  'f': 'rnbqkbnr/pppppppp/8/8/8/P7/1PPPPPPP/RNBQKBNR b KQkq',
  'id': 49,
  'm': 'a2a3',
  'n': 'Anderssen Opening'},
 {'c': 'A00',
  'f': 'rnbqkbnr/1ppppppp/8/p7/1P6/P7/2PPPPPP/RNBQKBNR b KQkq',
  'id': 50,
  'm': 'a2a3 a7a5 b2b4',
  'n': 'Anderssen Opening: Polish Gambit'},
 {'c': 'A00'

In [ ]:
# Load up the trained model
move_predictor = MovePredictor(cfg)
position_parser = ChessPositionParser(cfg, [], [])

elos = [1000, 3000]
time_control = '3600+30'
num_simulations = 50

results = []

mcts = ChessMonteCarloTreeSearch(cfg, move_predictor, position_parser, num_simulations=num_simulations)

count = 0
total = len(ecos) * len(elos)
for opening in ecos:
    for elo in elos:
        fen = opening['f'] + ' - 0 1'
        ab_eval, mcts_eval = mcts.set_position(fen, player_elo=elo, opponent_elo=elo).run_search().get_evals()
        results.append({
            'eco': opening['c'],
            'fen': opening['f'],
            'moves': opening['m'],
            'name': opening['n'],
            'player_elo': elo,
            'ab_eval': ab_eval,
            'mcts_eval': mcts_eval,
        })
        
        count += 1
        print(f'{count} of {total}')
        



j = json.dumps(results, indent=2)

with open('/Users/tom/Projects/Portfolio/web-static/public/chess/openings-strength.json', 'w') as f:
    print(j, file=f)

1 of 6828
2 of 6828
3 of 6828
4 of 6828
5 of 6828
6 of 6828
7 of 6828
8 of 6828
9 of 6828
10 of 6828
11 of 6828
12 of 6828
13 of 6828
14 of 6828
15 of 6828
16 of 6828
17 of 6828
18 of 6828
19 of 6828
20 of 6828
21 of 6828
22 of 6828
23 of 6828
24 of 6828
25 of 6828
26 of 6828
27 of 6828
28 of 6828
29 of 6828
30 of 6828
31 of 6828
32 of 6828
33 of 6828
34 of 6828
35 of 6828
36 of 6828
37 of 6828
38 of 6828
39 of 6828
40 of 6828
41 of 6828
42 of 6828
43 of 6828
44 of 6828
45 of 6828
46 of 6828
47 of 6828
48 of 6828
49 of 6828
50 of 6828
51 of 6828
52 of 6828
53 of 6828
54 of 6828
55 of 6828
56 of 6828
57 of 6828
58 of 6828
59 of 6828
60 of 6828
61 of 6828
62 of 6828
63 of 6828
64 of 6828
65 of 6828
66 of 6828
67 of 6828
68 of 6828
69 of 6828
70 of 6828
71 of 6828
72 of 6828
73 of 6828
74 of 6828
75 of 6828
76 of 6828
77 of 6828
78 of 6828
79 of 6828
80 of 6828
81 of 6828
82 of 6828
83 of 6828
84 of 6828
85 of 6828
86 of 6828
87 of 6828
88 of 6828
89 of 6828
90 of 6828
91 of 6828
92 of 68